In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Subset
from torch import Tensor
from typing import Tuple, Callable
from itertools import chain
import copy
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import GradScaler
from torch.distributions import MultivariateNormal

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

scaler = GradScaler(device)

Using cuda device


/tmp/ipykernel_29824/3266725043.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(device)


In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root='../data',
    train=True,
    # download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root='../data',
    train=False,
    # download=True,
    transform=ToTensor(),
)

In [4]:
target_label = 0
training_data_incides = torch.where(training_data.targets == target_label)[0]
test_data_incides = torch.where(test_data.targets == target_label)[0]

training_data = Subset(training_data, training_data_incides)
test_data = Subset(test_data, test_data_incides)

In [5]:
# Define validation split fraction
val_fraction = 0.1
dataset_size = len(training_data)
val_size = int(val_fraction * dataset_size)
train_size = dataset_size - val_size

# Create a reproducible shuffled list of indices
generator = torch.Generator().manual_seed(42)
indices = torch.randperm(dataset_size, generator=generator).tolist()

# Split indices for train and validation
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Wrap Subsets for train and validation datasets
train_data = Subset(training_data, train_indices)
val_data = Subset(training_data, val_indices)

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

print(f"Training batches: {len(train_dataloader)}")
print(f"Test batches: {len(test_dataloader)}")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Training batches: 94
Test batches: 16


$$
z_B = \exp\left(-s(z_A)\right) \odot \left(x_B - b(z_A)\right)  
$$

$$
J = 
\begin{bmatrix}
I_d & 0 \\
\frac{\partial z_B}{\partial x_A} & \mathrm{diag}\big(\exp(-s)\big)
\end{bmatrix}
$$

$$
x_B = \exp\big(s(z_A, w)\big) \odot z_B + b(z_A, w)
$$

In [7]:
class CouplingLayer(nn.Module):
    def __init__(
        self,
        split_at: int,
        scale_net: nn.Module, # s
        shift_net: nn.Module, # b
        alternate_parts: bool = False
    ) -> None:
        super().__init__()
        self.split_at = split_at
        self.scale_net = scale_net
        self.shift_net = shift_net
        self.alternate_parts = alternate_parts
    

    def _split(self, x: Tensor) -> Tuple[Tensor, Tensor]:
        if self.alternate_parts:
            return x[:, self.split_at:], x[:, :self.split_at]
        else:
            return x[:, :self.split_at], x[:, self.split_at:]


    def _merge(self, xA: Tensor, xB: Tensor) -> Tensor:
        if self.alternate_parts:
            return torch.cat((xB, xA), dim=1)
        else:
            return torch.cat((xA, xB), dim=1)


    def _get_scale_and_shift(self, zA: Tensor) -> Tuple[Tensor, Tensor]:
        log_scale = self.scale_net(zA)
        log_scale = torch.clamp(log_scale, min=-5, max=3)
        shift = self.shift_net(zA)
        return log_scale, shift


    def forward(self, x: Tensor, log_det_total: Tensor) -> Tuple[Tensor, Tensor]:
        xA, xB = self._split(x)
        zA = xA
        log_scale, shift = self._get_scale_and_shift(zA)

        zB = torch.exp(-log_scale) * (xB - shift)
        z = self._merge(zA, zB)

        log_det_current = -torch.sum(log_scale, dim=1)
        log_det_total = log_det_total + log_det_current
        return z, log_det_total


    def inverse(self, z: Tensor) -> Tensor:
        zA, zB = self._split(z)
        xA = zA
        log_scale, shift = self._get_scale_and_shift(zA)

        xB = torch.exp(log_scale) * (zB + shift)
        x = self._merge(xA, xB)
        return x

In [ ]:
class MLP(nn.Module):
    def __init__(self,
        layers_dims: list,
        activation_layer: nn.Module = nn.ReLU(),
        bias: bool = True,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__()
        assert len(layers_dims) > 2
        layers = []
        
        for in_features, out_features in zip(layers_dims[:-2], layers_dims[1:-1]):
            layers.append(nn.Linear(in_features, out_features, bias=bias, device=device, dtype=dtype))
            layers.append(copy.deepcopy(activation_layer))
        
        layers.append(nn.Linear(layers_dims[-2], layers_dims[-1], bias=bias, device=device, dtype=dtype))

        # Init last layer to small values for stability
        nn.init.zeros_(layers[-1].weight)
        if bias:
            nn.init.zeros_(layers[-1].bias)
            
        self.model = nn.Sequential(*layers)
        

    def forward(self, x) -> Tensor:
        return self.model(x)
        

In [ ]:
class LambdaLayer(nn.Module):
    def __init__(
        self,
        func: Callable[[Tensor], Tensor],
        inv_func: Callable[[Tensor], Tensor],
    ) -> None:
        super().__init__()
        self.func = func
        self.inv_func = inv_func

    def forward(self, x: Tensor, *args, **kwargs):
        y = self.func(x)
        if args or kwargs:
            return (y, *args) if not kwargs else (y, *args, kwargs)
        return y
    
    def inverse(self, x: Tensor) -> Tensor:
        return self.inv_func(x)


def vec_to_img(x: Tensor) -> Tensor:
    return x.view(-1, 28, 28)

def img_to_vec(x: Tensor) -> Tensor:
    return x.flatten(1)


def sigmoid_inverse(x: Tensor):
    # Avoid log(0)
    x = torch.clamp(x, 1e-6, 1 - 1e-6)
    x = torch.log(x) - torch.log(1 - x)
    return x

def add_noise(x: Tensor):
    # Add uniform noise for dequantization (important for discrete data)
    return x + torch.rand_like(x) / 256.0


In [ ]:
class FlowSequential(nn.Module):
    def __init__(self, data_dim: int, layers: nn.Module):
        super().__init__()
        self.layers = nn.ModuleList(layers)

        # Check if all modules implement required interface
        for i, module in enumerate(self.layers):
            if not hasattr(module, 'forward') or not hasattr(module, 'inverse'):
                raise TypeError(f"Module at index {i} must implement both 'forward' and 'inverse' methods.")
            
        self.register_buffer('base_mean', torch.zeros(data_dim))
        self.register_buffer('base_cov', torch.eye(data_dim))

    def forward(
        self,
        x: torch.Tensor,
        log_det: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        if log_det is None:
            log_det = torch.zeros(len(x), device=x.device)
        for module in self.layers:
            x, log_det = module(x, log_det)
        return x, log_det

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        for module in reversed(self.layers):
            z = module.inverse(z)
        return z
    
    def __call__(
        self,
        x: torch.Tensor,
        log_det: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # Optional: put extra logic here before/after forward
        return self.forward(x, log_det)
    
    def sample(self, num_samples: int, device:str=device) -> Tensor:
        base_dist = MultivariateNormal(self.base_mean, self.base_cov)
        z = base_dist.sample((num_samples,)).to(device)
        
        # Transform through inverse flow
        with torch.no_grad():
            x = self.inverse(z)
        
        return x
    
    def log_prob(self, x: Tensor):
        z, log_det = self.forward(x)
        
        base_dist = MultivariateNormal(self.base_mean, self.base_cov)
        log_prob_base = base_dist.log_prob(z)
        
        log_prob = log_prob_base + log_det
        return log_prob

In [ ]:
# For implementation challange
NORMAL_DIST_CONST = torch.log(torch.tensor(2 * torch.pi, device=device))
def normal_NLL(X: Tensor) -> Tensor:
    """Compute -log p(z) for standard Gaussian"""
    d = X.shape[1]
    const_term = (d / 2) * NORMAL_DIST_CONST.to(X.device)
    squared_term = 0.5 * X.pow(2).sum(dim=1)
    return const_term + squared_term


def flow_NLL_loss(
    z_sample: Tensor,
    total_log_det: Tensor,
) -> Tensor:
    """Flow loss = base NLL - total log det (averaged)"""
    return torch.mean(normal_NLL(z_sample) - total_log_det)

In [ ]:
def run_epoch(
        model: nn.Module,
        data_loader: DataLoader,
        device: str,
        optimizer,
        training_mode: bool = True,
    ) -> float:
    """
    Run a single epoch in either training or evaluation mode.
    
    Args:
        model: The model to train/evaluate
        dataloader: DataLoader for the dataset
        device: Device to run on (cuda/cpu)
        optimizer: Optimizer for training (required if train_mode=True)
        train_mode: If True, run in training mode; if False, run in evaluation mode
        max_grad_norm: Maximum gradient norm for clipping (only used in training)
    
    Returns:
        average_loss: Average loss for the epoch
    """
    total_loss, num_batches = 0, 0
    
    for data, _ in tqdm(data_loader, total=len(data_loader)):
        data = data.to(device)
        if training_mode:
            optimizer.zero_grad()
        
        # Compute negative log likelihood
        log_prob = model.log_prob(data)
        loss = -log_prob.mean()
        
        if training_mode:
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    return avg_loss
    

In [ ]:
def save_model_with_train_state(
    checkpoint_path: str,
    model: nn.Module,
    epoch: int,
    optimizer,
    scheduler,
    train_avg_loss: float,
    val_avg_loss: float,
    train_losses: list,
    val_losses: list,
):
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_avg_loss,
        'val_loss': val_avg_loss,
        'train_losses': train_losses,
        'val_losses': val_losses
    }, checkpoint_path)


def load_best_model(
        model: nn.Module,
        checkpoint_path: str,
        device: str
) -> nn.Module:
    """
    Load the best saved model from checkpoint.
    
    Args:
        model: The model architecture (should match the saved model)
        checkpoint_path: Path to the saved checkpoint
        device: Device to load the model on
        
    Returns:
        The loaded model
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Loaded model from epoch {checkpoint['epoch']} with validation loss: {checkpoint['val_loss']:.4f}")
    
    return model

In [ ]:
def visualize_samples(model, device, epoch=None):
    """Generate and visualize samples from the trained model"""
    # Sample
    model.eval()
    samples = model.sample(16, device).cpu()
    samples = torch.clamp(samples, 0, 1)
    
    # Plot samples
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    for i, ax in enumerate(axes.flat):
        ax.imshow(samples[i], cmap='gray')
        ax.axis('off')
    
    title = f'Generated Samples - Epoch {epoch}' if epoch else 'Generated Samples'
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    
    model.train()


In [ ]:
def train_flow(
        model: nn.Module,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device: str,
        num_epochs: int = 50,
        lr: float = 1e-4,
        patience: int = 10,
        save_path: str = "../models/best_model.pth"
    ) -> Tuple[list, list]:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    train_losses = []
    val_losses = []

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    best_epoch = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_avg_loss = run_epoch(
            model, train_dataloader, device, optimizer, True)

        
        model.eval()
        with torch.no_grad():
            val_avg_loss = run_epoch(
                model, val_dataloader, device, optimizer, False)
        
        scheduler.step()
        train_losses.append(train_avg_loss)
        val_losses.append(val_avg_loss)


        if val_avg_loss < best_val_loss:
            best_val_loss = val_avg_loss
            best_epoch = epoch + 1
            epochs_without_improvement = 0

            save_model_with_train_state(
                save_path, model, epoch, optimizer, scheduler,
                train_avg_loss, val_avg_loss, train_losses, val_losses,
            )
            
            print(f'Epoch {epoch+1} completed. Train Loss: {train_avg_loss:.4f}, '
                  f'Val Loss: {val_avg_loss:.4f} # NEW BEST MODEL #')
        else:
            epochs_without_improvement += 1
            print(f'Epoch {epoch+1} completed. Train Loss: {train_avg_loss:.4f}, '
                  f'Val Loss: {val_avg_loss:.4f} (Best: {best_val_loss:.4f} at epoch {best_epoch})')
        

        if (epoch + 1) % 10 == 0:
            visualize_samples(model, device, epoch+1)

        # Early stopping check
        if epochs_without_improvement >= patience:
            print(f'\nEarly stopping triggered after {patience} epochs without improvement.')
            print(f'Best validation loss: {best_val_loss:.4f} at epoch {best_epoch}')
            break

    return train_losses, val_losses

In [ ]:
flow_net = FlowSequential(
    784,
    [
        LambdaLayer(img_to_vec, vec_to_img),
        LambdaLayer(add_noise, lambda x: x),
        LambdaLayer(sigmoid_inverse, torch.sigmoid),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device)),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device), True),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device)),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device), True),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device)),
        CouplingLayer(392, MLP([392, 512, 512, 392], device = device), MLP([392, 512, 512, 392], device = device), True),
    ]

).to(device)


In [ ]:
# Test
x_sample = next(iter(train_dataloader))[0].to(device)
x_to_z = flow_net(x_sample, 0)

In [ ]:
model_save_path = '../models/coupling_model.pth'
train_losses, val_losses = train_flow(
    flow_net, train_dataloader, val_dataloader, device,
    num_epochs=10, lr=1e-4, save_path=model_save_path)